In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib

import IPython.display

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.interpolate
import scipy.signal

import pymedphys
import pymedphys._wlutz.iview

In [ ]:
indexed_dir = pathlib.Path(r'S:\DataExchange\iViewDB_decoded\indexed')
movie_dirs = list(indexed_dir.glob('*/*/*/*/*'))
movie_dirs

In [ ]:
wlutz_results = {}

In [ ]:
edge_lengths = [20, 24]
pd.set_option("display.max_rows", 101)

In [ ]:
for directory in movie_dirs:
    image_paths = list(directory.glob('*.png'))
    print(image_paths)
    wlutz_results[directory] = pymedphys._wlutz.iview.batch_process(image_paths, edge_lengths, display_figure=True)
    
    IPython.display.display(wlutz_results[directory])

In [ ]:
for directory in movie_dirs:
    try:
        wlutz_results[directory]
    except KeyError:
        image_paths = list(directory.glob('*.png'))
        print(image_paths)
        
        try:
            wlutz_results[directory] = pymedphys._wlutz.iview.batch_process(image_paths, edge_lengths, display_figure=True)
            IPython.display.display(wlutz_results[directory])
        except ValueError:
            continue

In [ ]:
for directory in movie_dirs:
    try:
        wlutz_results[directory]
    except KeyError:
        image_paths = list(directory.glob('*.png'))
        print(image_paths)
        
        try:
            wlutz_results[directory] = pymedphys._wlutz.iview.batch_process(image_paths, edge_lengths, display_figure=True)
            IPython.display.display(wlutz_results[directory])
        except ValueError:
            continue

In [ ]:
for directory in movie_dirs:
    try:
        wlutz_results[directory]
    except KeyError:
        image_paths = list(directory.glob('*.png'))
        print(image_paths)
        
        try:
            wlutz_results[directory] = pymedphys._wlutz.iview.batch_process(image_paths, edge_lengths, display_figure=True)
            IPython.display.display(wlutz_results[directory])
        except ValueError:
            continue

In [ ]:
for key, table in wlutz_results.items():
    print(key)
    IPython.display.display(table)

In [ ]:
keys = list(wlutz_results.keys())
keys

In [ ]:
direction_keys = [
    key.parent.stem for key in keys
]

direction_keys

In [ ]:
rotations = [
    wlutz_results[key]['Rotation']
    for key in keys
]




lt_zero = rotations[0] < 0

gantry = np.empty_like(rotations[0])
gantry[lt_zero] = -180 - rotations[0][lt_zero]

gte_zero = np.invert(lt_zero)

gantry[gte_zero] = 180 - rotations[0][gte_zero]

gantry

In [ ]:
gantry = []

for i, direction_key in enumerate(direction_keys):

    if direction_keys[i] == '00_CW':
        diff = np.diff(np.concatenate([[-180], rotations[i]]))
        diff[diff > 0] = diff[diff > 0] - 180

        gantry.append(-180 - np.cumsum(diff * 2))
    elif direction_keys[i] == '01_CC':
        diff = np.diff(np.concatenate([[0], rotations[i]]))
        diff[diff < 0] = diff[diff < 0] + 180

        gantry.append(180 - np.cumsum(diff * 2))
    else:
        raise ValueError("Expected one of '00_CW' or '01_CC'")


    
gantry

In [ ]:
bb_x = [
    wlutz_results[key]['BB x'] for key in keys
]
bb_y = [
    wlutz_results[key]['BB y'] for key in keys
]

In [ ]:
gantry

In [ ]:
bb_x

In [ ]:
scipy.interpolate.interp1d?

In [ ]:
interp_bb_x = [
    scipy.interpolate.interp1d(g, x, bounds_error=False, fill_value='extrapolate')
    for g, x in zip(gantry, bb_x)
]

def get_avg_bb_x(gantry):
    results = []
    
    for interp in interp_bb_x:
        results.append(interp(gantry))
        
    return (np.min(results, axis=0) + np.max(results, axis=0))/2


interp_bb_y = [
    scipy.interpolate.interp1d(g, y, bounds_error=False, fill_value='extrapolate')
    for g, y in zip(gantry, bb_y)
]

def get_avg_bb_y(gantry):
    results = []
    
    for interp in interp_bb_y:
        results.append(interp(gantry))
    
    return (np.min(results, axis=0) + np.max(results, axis=0))/2

get_avg_bb_y([0, 2])

In [ ]:
# gantry_all = np.concatenate(gantry)
# ind = np.argsort(gantry_all)
# sorted_gantry = gantry_all[ind]

# within_bounds = np.logical_and(sorted_gantry <= 180, sorted_gantry >= -180)
# sorted_gantry = sorted_gantry[within_bounds]

# sorted_bb_x = np.concatenate(bb_x)[ind][within_bounds]
# sorted_bb_y = np.concatenate(bb_y)[ind][within_bounds]

In [ ]:
# b, a = scipy.signal.butter(3, 0.05)
# filtered_bb_x = scipy.signal.filtfilt(b, a, sorted_bb_x)
# filtered_bb_y = scipy.signal.filtfilt(b, a, sorted_bb_y)

In [ ]:
# plt.plot(sorted_gantry, filtered_bb_x)

In [ ]:
# unique_gantry, unique_inverse = np.unique(sorted_gantry, return_inverse=True)
# inc = np.arange(len(unique_inverse))

# make_unique = np.ones((len(unique_gantry), len(unique_inverse))) * np.nan
# make_unique[unique_inverse, inc] = sorted_bb_x
# striclty_increasing_bb_x = np.nanmean(make_unique, axis=1)

# make_unique[unique_inverse, inc] = sorted_bb_y
# striclty_increasing_bb_y = np.nanmean(make_unique, axis=1)

In [ ]:
# def predict_bb_pos(gantry, gantry_range=10):
#     gantry = np.array(gantry)
    
#     lte = gantry[:,None] - gantry_range <= gantry_all[None,:]
#     gte = gantry[:,None] + gantry_range >= gantry_all[None,:]
#     in_range = np.logical_and(lte, gte)
    
#     sorted_bb_x
    
#     return in_range
    
    
# predict_bb_pos([0, 1], gantry_range=10)

In [ ]:
# unique_gantry = np.unique(sorted_gantry)
# bb_x_interp = scipy.interpolate.interp1d(sorted_gantry, filtered_bb_x, bounds_error=False)
# bb_y_interp = scipy.interpolate.interp1d(sorted_gantry, filtered_bb_y, bounds_error=False)

In [ ]:
# bb_x_interp = scipy.interpolate.UnivariateSpline(unique_gantry, strictly_increasing_bb_x, s=0.1)
# bb_y_interp = scipy.interpolate.UnivariateSpline(unique_gantry, strictly_increasing_bb_y, s=1)

gantry_i = np.linspace(-180, 180, 91)

In [ ]:
for i, key in enumerate(keys):   
    plt.plot(gantry[i], bb_x[i], '.')
    
plt.plot(gantry_i, get_avg_bb_x(gantry_i))
plt.xlim([-180, 180])

In [ ]:
for i, key in enumerate(keys):
    plt.plot(gantry[i], bb_y[i], '.')

plt.plot(gantry_i, get_avg_bb_y(gantry_i))
plt.xlim([-180, 180])